In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"

execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb;reload(pu_delayfb)
import analysis.lib.fastcarboncontrol.fcc as fcc; reload(fcc)

from analysis.lib.fitting import common, fit; reload(common); reload(fit)

import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline

In [ ]:
reload(pu_delayfb)

det6_ts = 20170531141332
det12_ts  = 20170531142610

det6_x, det6_y, det6_y_u, det6_fr = pu_delayfb.calibrate_LDE_phase(
    contains="phase_fb_delayline", older_than=str(det6_ts + 1), do_fit=True, ret_data_fit=True,
    fixed=[0,1]
)

det12_x, det12_y, det12_y_u, det12_fr = pu_delayfb.calibrate_LDE_phase(
    contains="phase_fb_delayline", older_than=str(det12_ts + 1), do_fit=True, ret_data_fit=True,
    fixed=[0,1]
)

In [ ]:
plt.figure()

eb6 = plt.errorbar(det6_x, det6_y, det6_y_u, fmt='o', label=r'$6\degree$ over-rotation')
eb12 = plt.errorbar(det12_x, det12_y, det12_y_u, fmt='o', label=r'$12\degree$ over-rotation')

def avg_param(param):
    return (det6_fr['params_dict'][param] + det12_fr['params_dict'][param]) / 2.0

avg_T = avg_param('t')
avg_phi = avg_param('phi')
avg_A = avg_param('A')

_, ff_6, _ = common.fit_decaying_cos(6./360., 0.0, avg_A, avg_phi, avg_T)
_, ff_12, _ = common.fit_decaying_cos(12./360., 0.0, avg_A, avg_phi, avg_T)

plot_xvals = np.linspace(0,60,1000)

plt.plot(plot_xvals, ff_6(plot_xvals), color=eb6[0].get_color())
plt.plot(plot_xvals, ff_12(plot_xvals), color=eb12[0].get_color())

plt.xlim(0, 60)
plt.ylim(-1, 1)
plt.legend(loc=1)
plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Nuclear expectation value $\langle X \rangle$")
plt.savefig("thesis-plots/phase-control-detuning.png", dpi=300)

In [ ]:
avg_phi

In [ ]:
2/360.0/443e3